## Apply Quantum SVM with PCA and 4-qubits XYZ ansatz

### 0. Load libraries

In [ ]:
import os
from os import listdir
import pandas as pd
from collections import Counter

import torch
import torch.nn as nn
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

import pennylane as qml
from pennylane import numpy as np

import data
import embedding

### 1. Load dataset

In [ ]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('COVID19', feature_reduction, classes)

In [ ]:
print("X_train:",X_train.shape,"/X_test:",X_test.shape,"/Y_train:",Y_train.shape,"/Y_test:",Y_test.shape)

In [ ]:
print(Counter(Y_train), Counter(Y_test))

### 2. Apply PCA

In [ ]:
full_X = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_test)], axis=0)
print("Before PCA:",full_X.shape)

pca_XYZ = PCA(n_components=15)
fixed_X_XYZ = pca_XYZ.fit_transform(X=full_X)
print("After PCA:",fixed_X_XYZ.shape)

fixed_X_XYZ_train = fixed_X_XYZ[:98,:]
fixed_X_XYZ_test = fixed_X_XYZ[98:,:]

X_XYZ_train=[]
X_XYZ_test=[]
for x in fixed_X_XYZ_train:
    X_XYZ_train.append(x / 2)
for x in fixed_X_XYZ_test:
    X_XYZ_test.append(x / 2)

X_XYZ_train = np.array(X_XYZ_train)
X_XYZ_test = np.array(X_XYZ_test)

print("(XYZ) :", fixed_X_XYZ_train.shape, fixed_X_XYZ_test.shape)

### 3. Define embedding circuits for encoding methods

In [ ]:
dev = qml.device('default.qubit', wires=8)

In [ ]:
@qml.qnode(dev, interface="torch")
def XYZ_circuit(inputs):
    embedding.repeat_XYZ_with_inverse_8_qubits(input1=inputs[:15], input2=inputs[15:])
    return qml.probs(wires=range(8))
    
XYZ_qlayer = qml.qnn.TorchLayer(XYZ_circuit, weight_shapes={})

def XYZ_kernel(x1, x2):
    x1 = x1.view(1, -1)
    x2 = x2.view(1, -1)
    concatted_x = torch.concat([x1, x2], 1)
    x = XYZ_qlayer(concatted_x)
    kernel_value = x[:,0].item()
    return kernel_value

### 4. Define a kernel matrices from new ansatz

In [ ]:
def kernel_matrix_XYZ(A, B):
    return np.array([[XYZ_kernel(torch.tensor(a).float(), torch.tensor(b).float()) for b in B] for a in A])

### 5. Apply SVM classifier

In [ ]:
svm_XYZ = SVC(kernel=kernel_matrix_XYZ, class_weight='balanced').fit(X_XYZ_train, Y_train)

### 6. Evaluate trained SVM with XYZ ansatz by test dataset

#### 6-1) SVM with XYZ

In [ ]:
accs =[]
for i in range(5):
    predictions = svm_XYZ.predict(X_XYZ_test)
    print("predictions:", predictions)
    print("true:", Y_test)
    
    acc = balanced_accuracy_score(Y_test, predictions)
    accs.extend([acc]) 

print("Accuracy with PCA and the kernel matrix of the XYZ ansatz:", np.mean(accs),'±',np.std(accs))